In [13]:
from PIL import Image
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Padding
def pad_one_im(direc_in, name, direc_out, pad_n):
    img_file = direc_in + name
    img = Image.open(img_file)
    old_size = img.size
    new_size = (old_size[0] + pad_n, old_size[1]+pad_n)
    new_im = Image.new("RGB", new_size)
    new_im.paste(img, ((new_size[0] - old_size[0])/2, (new_size[1]-old_size[1])/2))
    out_file = img_file[0:(len(img_file)-4)] + "padded.png"
    new_im.save(direc_out + out_file)
    
def pad_directory(direc_in, direc_out, pad_n):
    imgs = os.listdir(direc_in)
    for i in imgs:
        if '.DS_Store' not in i:
            pad_one_im(direc_in, i, direc_out, pad_n)

In [ ]:
#Black & White
def makeItGray(direc_in, name, direc_out):
    img_file = direc_in + name
    img = Image.open(img_file)
    gray = img.convert('LA')
    out_file = img_file[0:(len(img_file)-4)] + "_gray.png" 
    gray.save(direc_out + out_file)
    
def makeDirGray(direc_in, direc_out):
    imgs = os.listdir(direc_in)
    for i in imgs:
        if '.DS_Store' not in i:
            makeItGray(direc_in, i, direc_out)

In [ ]:
#Color Balancing

In [ ]:
#Luminance Balancing

In [ ]:
direc_in_list = ['../data/raw/train' + d for d in os.listdir('../data/raw/train')
                if '.DS_Store' not in d]

In [20]:
direc_gray_out_list = ['../data/processed/gray/'+ d for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]

makeDirGray(direc_in_list, direc_gray_out_list)

['../data/processed/gray/ALB', '../data/processed/gray/BET', '../data/processed/gray/DOL', '../data/processed/gray/LAG', '../data/processed/gray/NoF', '../data/processed/gray/OTHER', '../data/processed/gray/SHARK', '../data/processed/gray/YFT']


In [ ]:
padded_out_list = ['../data/processed/padded/'+ d for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]

pad_directory(direc_in_list, padded_out_list, 50)

In [ ]:
gray_in_list = direc_gray_out_list

padded_gray_out_list = ['../data/processed/gray_padded/'+ d for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]

pad_directory(gray_in_list, padded_gray_out_list, 50)